In [52]:
from corus import load_lenta
import pandas as pd

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
# next(records)


Загрузка данных. Берем максимум 100000 примеров для ускорения обработки и обучения.


In [53]:
records = load_lenta(path)
data = []
for record in records:
    if record.topic is None:
        continue
    data.append({
        'title': record.title,
        'text': record.text,
        'topic': record.topic
    })
    if len(data) >= 100_000:
        break
df = pd.DataFrame(data)

Предобработка текста. Текст из колонок title и text объединяется в одну колонку full_text, после чего каждый текст проходит через несколько этапов обработки: приведение к нижнему регистру, удаление символов, не являющихся буквами или пробелами, токенизация (разбиение на слова), удаление стоп-слов и коротких токенов, а также лемматизация (приведение слов к их нормальной форме). Обработанные тексты сохраняются в новой колонке processed_text. Для ускорения обработки используется библиотека swifter с прогресс-баром. Путем проб обнаружено, что лемматизация дает наилучший результат, хотя разница со стеммингом или вообще без какого-либо из двух способов минимальна.

In [54]:
import re
import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from functools import lru_cache
import swifter

nltk.download('punkt')
nltk.download('stopwords')

morph = pymorphy2.MorphAnalyzer()
stemmer = SnowballStemmer("russian")

stop_words = set(stopwords.words('russian'))

df['full_text'] = df['title'] + ' ' + df['text']


@lru_cache(maxsize=100000)
def lemmatize_word(word):
    return morph.parse(word)[0].normal_form

def preprocess_text(text):
    text = re.sub(r'[^а-яё\s]|\d+', '', text.lower())
    tokens = word_tokenize(text, language = "russian")
    tokens = [lemmatize_word(t) for t in tokens if t not in stop_words and len(t) > 2] # Лемматизация
    # tokens = [stemmer.stem(t) for t in tokens  if t not in stop_words and len(t) > 2]  # Стемминг
    return ' '.join(tokens)

# Применение с распараллеливанием
df['processed_text'] = df['full_text'].swifter.progress_bar(True).apply(preprocess_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rustam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rustam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

In [55]:
# Посчитаем частоту каждого топика
topic_counts = df['topic'].value_counts()
print(topic_counts)

# Удалим классы с менее чем 2 экземплярами
valid_topics = topic_counts[topic_counts >= 2].index
df_filtered = df[df['topic'].isin(valid_topics)].copy()

topic
Россия               15151
Мир                  14421
Спорт                10045
Экономика             7682
Интернет и СМИ        6935
Силовые структуры     6925
Бывший СССР           6810
Культура              6578
Наука и техника       5645
Из жизни              4903
Ценности              4480
Дом                   3408
Путешествия           3223
Бизнес                1993
69-я параллель         815
Крым                   661
Культпросвет           307
                        17
Оружие                   1
Name: count, dtype: int64


Разделяем датасет на обучающую, валидационную и тестовую выборки со стратификацией в пропорции 60/20/20. В качестве целевой переменной используем атрибут topic.

In [56]:
from sklearn.model_selection import train_test_split

X = df_filtered['processed_text']
y = df_filtered['topic']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

Замеряем базовое качество с dummy-бейзлайном 

In [63]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy = DummyClassifier(strategy='most_frequent', random_state=42)
dummy.fit(X_train, y_train)
y_pred_dummy = dummy.predict(X_val)
print(f'Baseline accuracy: {accuracy_score(y_val, y_pred_dummy):.4f}')

Baseline accuracy: 0.1515


Пайплайн включает сравнение двух подходов векторизации текста (CountVectorizer и TfidfVectorizer) в сочетании с LogisticRegression для классификации. Обучаются две модели на тренировочных данных, после чего оценивается их точность на валидационной выборке.

Цель — определить, какой подход (CountVectorizer или TfidfVectorizer) лучше подходит для задачи.

In [67]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Модель с CountVectorizer
pipe_count = Pipeline([
    ('vec', CountVectorizer(max_features=20_000)),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])
pipe_count.fit(X_train, y_train)
y_pred_count = pipe_count.predict(X_val)
print(f'CountVectorizer accuracy: {accuracy_score(y_val, y_pred_count):.4f}')

# Модель с TfidfVectorizer
pipe_tfidf = Pipeline([
    ('vec', TfidfVectorizer(max_features=20_000)),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])
pipe_tfidf.fit(X_train, y_train)
y_pred_tfidf = pipe_tfidf.predict(X_val)
print(f'TfidfVectorizer accuracy: {accuracy_score(y_val, y_pred_tfidf):.4f}')

CountVectorizer accuracy: 0.8376
TfidfVectorizer accuracy: 0.8377


Используем GridSearchCV для подбора гиперпараметров TfidfVectorizer и LogisticRegression. Перебираем диапазон n-грамм, количество признаков и параметр регуляризации C. Кросс-валидация на 3 фолдах, параллелизация на всех ядрах (n_jobs=-1). После обучения выводим лучшие параметры и точность.

In [69]:
from sklearn.model_selection import GridSearchCV

params = {
    'vec__ngram_range': [(1, 1), (1, 2)],
    'vec__max_features': [10_000, 20_000, 30_000],
    'clf__C': [0.1, 1, 10]
}

grid = GridSearchCV(pipe_tfidf, params, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)
print(f'Best params: {grid.best_params_}')
print(f'Best CV accuracy: {grid.best_score_:.4f}')

Best params: {'clf__C': 10, 'vec__max_features': 30000, 'vec__ngram_range': (1, 2)}
Best CV accuracy: 0.8484


Оценка качества лучшего пайплайна на отложенной выборке 

In [70]:
best_model = grid.best_estimator_
y_test_pred = best_model.predict(X_test)
print(f'Test accuracy: {accuracy_score(y_test, y_test_pred):.4f}')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred, target_names=df_filtered['topic'].unique(), zero_division=0))

Test accuracy: 0.8579
                   precision    recall  f1-score   support

           Россия       0.00      0.00      0.00         3
            Спорт       0.93      0.73      0.82       163
      Путешествия       0.66      0.54      0.59       399
              Мир       0.89      0.88      0.89      1362
      Бывший СССР       0.88      0.84      0.86       681
   Интернет и СМИ       0.82      0.79      0.81       980
Силовые структуры       0.85      0.81      0.83      1387
        Экономика       0.81      0.77      0.79       132
         Культура       0.71      0.39      0.50        62
              Дом       0.88      0.90      0.89      1316
  Наука и техника       0.86      0.90      0.88      2884
         Из жизни       0.90      0.90      0.90      1129
         Ценности       0.88      0.83      0.86       645
           Бизнес       0.81      0.85      0.83      3031
   69-я параллель       0.79      0.78      0.78      1385
                        0.97     